# Amazon SageMaker Batch Transform: Associate prediction results with their corresponding input records


_**Use SageMaker's XGBoost to train a binary classification model and for a list of tumors in batch file, predict if each is malignant**_

_**It also shows how to use the input output joining / filter feature in Batch transform in details**_

---



## Background
This purpose of this notebook is to train a model using SageMaker's XGBoost and UCI's breast cancer diagnostic data set to illustrate at how to run batch inferences and how to use the Batch Transform I/O join feature. UCI's breast cancer diagnostic data set is available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give training and batch transform access to your data. The snippet below will use the same role used by your SageMaker notebook instance. Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.

In [3]:
!pip3 install -U sagemaker

  Using cached sagemaker-2.221.1-py3-none-any.whl.metadata (14 kB)
Using cached sagemaker-2.221.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.215.0
    Uninstalling sagemaker-2.215.0:
      Successfully uninstalled sagemaker-2.215.0


In [4]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "DEMO-breast-cancer-prediction-xgboost-highlevel"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


---
## Data sources

> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

## Data preparation


Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [5]:
import pandas as pd
import numpy as np

s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file(
    f"sagemaker-example-files-prod-{region}", "datasets/tabular/breast_cancer/wdbc.csv", filename
)
data = pd.read_csv(filename, header=None)

# specify columns extracted from wbdc.names
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
260,887549,M,20.310,27.06,132.90,1288.0,0.10000,0.10880,0.151900,0.093330,...,24.33,39.16,162.30,1844.0,0.15220,0.29450,0.37880,0.16970,0.3151,0.07999
459,9112712,B,9.755,28.20,61.68,290.9,0.07984,0.04626,0.015410,0.010430,...,10.67,36.92,68.03,349.9,0.11100,0.11090,0.07190,0.04866,0.2321,0.07211
319,894335,B,12.430,17.00,78.60,477.3,0.07557,0.03454,0.013420,0.016990,...,12.90,20.21,81.76,515.9,0.08409,0.04712,0.02237,0.02832,0.1901,0.05932
195,875878,B,12.910,16.33,82.53,516.4,0.07941,0.05366,0.038730,0.023770,...,13.88,22.00,90.81,600.6,0.10970,0.15060,0.17640,0.08235,0.3024,0.06949
154,8711002,B,13.150,15.34,85.31,538.9,0.09384,0.08498,0.092930,0.034830,...,14.77,20.50,97.67,677.3,0.14780,0.22560,0.30090,0.09722,0.3849,0.08633
335,89742801,M,17.060,21.00,111.80,918.6,0.11190,0.10560,0.150800,0.099340,...,20.99,33.15,143.20,1362.0,0.14490,0.20530,0.39200,0.18270,0.2623,0.07599
232,88203002,B,11.220,33.81,70.79,386.8,0.07780,0.03574,0.004967,0.006434,...,12.36,41.78,78.44,470.9,0.09994,0.06885,0.02318,0.03002,0.2911,0.07307
554,924632,B,12.880,28.92,82.50,514.3,0.08123,0.05824,0.061950,0.023430,...,13.89,35.74,88.84,595.7,0.12270,0.16200,0.24390,0.06493,0.2372,0.07242


#### Key observations:
* The data has 569 observations and 32 columns.
* The first field is the 'id' attribute that we will want to drop before batch inference and add to the final inference output next to the probability of malignancy.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features that we will use for training and inferencing.

Let's replace the M/B diagnosis with a 1/0 boolean value. 

In [6]:
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
361,901041,0,13.300,21.57,85.24,546.1,0.08582,0.06373,0.03344,0.02424,...,14.20,29.20,92.94,621.2,0.11400,0.1667,0.1212,0.05614,0.2637,0.06658
57,857793,1,14.710,21.59,95.55,656.9,0.11370,0.13650,0.12930,0.08123,...,17.87,30.70,115.70,985.5,0.13680,0.4290,0.3587,0.18340,0.3698,0.10940
527,91813702,0,12.340,12.27,78.94,468.5,0.09003,0.06307,0.02958,0.02647,...,13.61,19.27,87.22,564.9,0.12920,0.2074,0.1791,0.10700,0.3110,0.07592
121,86517,1,18.660,17.12,121.40,1077.0,0.10540,0.11000,0.14570,0.08665,...,22.25,24.90,145.40,1549.0,0.15030,0.2291,0.3272,0.16740,0.2894,0.08456
223,8812877,1,15.750,20.25,102.60,761.3,0.10250,0.12040,0.11470,0.06462,...,19.56,30.29,125.90,1088.0,0.15520,0.4480,0.3976,0.14790,0.3993,0.10640
180,873592,1,27.220,21.87,182.10,2250.0,0.10940,0.19140,0.28710,0.18780,...,33.12,32.85,220.80,3216.0,0.14720,0.4034,0.5340,0.26880,0.2856,0.08082
402,904689,0,12.960,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,...,14.13,24.61,96.31,621.9,0.09329,0.2318,0.1604,0.06608,0.3207,0.07247
110,864033,0,9.777,16.99,62.50,290.2,0.10370,0.08404,0.04334,0.01778,...,11.05,21.47,71.68,367.0,0.14670,0.1765,0.1300,0.05334,0.2533,0.08468


Let's split the data as follows: 80% for training, 10% for validation and let's set 10% aside for our batch inference job. In addition, let's drop the 'id' field on the training set and validation set as 'id' is not a training feature. For our batch set however, we keep the 'id' feature. We'll want to filter it out prior to running our inferences so that the input data features match the ones of training set and then ultimately, we'll want to join it with inference result. We are however dropping the diagnosis attribute for the batch set since this is what we'll try to predict.

In [7]:
# data split in three sets, training, validation and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list].drop(["id"], axis=1)
data_val = data[val_list].drop(["id"], axis=1)
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

Let's upload those data sets in S3

In [8]:
train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)
sess.upload_data(train_file, key_prefix="{}/train".format(prefix))

validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)
sess.upload_data(validation_file, key_prefix="{}/validation".format(prefix))

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
sess.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
sess.upload_data(batch_file_noID, key_prefix="{}/batch".format(prefix))

's3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/batch/batch_data_noID.csv'

---

## Training job and model creation

The below cell uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick off the training job using both our training set and validation set. Not that the objective is set to 'binary:logistic' which trains a model to output a probability between 0 and 1 (here the probability of a tumor being malignant).

In [9]:
%%time
from time import gmtime, strftime

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
)

sm_estimator.set_hyperparameters(
    objective="binary:logistic",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)

train_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

# Start training by calling the fit method in the estimator
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2024-06-01-21-50-54


2024-06-01 21:50:55 Starting - Starting the training job...
2024-06-01 21:51:09 Starting - Preparing the instances for training...
2024-06-01 21:51:36 Downloading - Downloading input data...
2024-06-01 21:52:06 Downloading - Downloading the training image...
2024-06-01 21:52:41 Training - Training image download completed. Training in progress..[2024-06-01 21:52:52.328 ip-10-0-151-130.ec2.internal:8 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-01 21:52:52.351 ip-10-0-151-130.ec2.internal:8 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-06-01:21:52:52:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-01:21:52:52:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-06-01:21:52:52:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:21:52:52:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-06-01:21:52:52:INFO] Determined 0 GPU(s) available on the insta

---

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.




#### 1. Create a transform job with the default configurations
Let's first skip these 3 new attributes and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [10]:
%%time

sm_transformer = sm_estimator.transformer(1, "ml.m4.xlarge")

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  # use input data without ID column
sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")
sm_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-06-01-21-53-36-967
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-01-21-53-37-617


..........................................
[2024-06-01:22:00:35:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:00:35:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:00:35:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
[2024-06-01:22:00:35:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:00:35:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:00:35:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {


Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [11]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [12]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file_noID)
output_df.head(8)

,0
0,0.993089
1,0.991388
2,0.993089
3,0.959429
4,0.985463
5,0.993089
6,0.993089
7,0.005192


#### 2. Join the input and the prediction results 
Now, let's associate the prediction results with their corresponding input records. We can also use the __input_filter__ to exclude the ID column easily and there's no need to have a separate file in S3.

* Set __input_filter__ to "$[1:]": indicates that we are excluding column 0 (the 'ID') before processing the inferences and keeping everything from column 1 to the last column (all the features or predictors)  
  
  
* Set __join_source__ to "Input": indicates our desire to join the input data with the inference results  

* Leave __output_filter__ to default ('$'), indicating that the joined input and inference results be will saved as output.

In [13]:
# content_type / accept and split_type / assemble_with are required to use IO joining feature
sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data with ID column cause InputFilter will filter it out
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-01-22-01-12-670


........................................[2024-06-01:22:07:55:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:07:55:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:07:55:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-06-01 22:07:55 +0000] [19] [INFO] Star

Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [14]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,844359,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368,0.993089
1,84610002,15.78,17.89,103.60,781.0,0.09710,0.12920,0.09954,0.06606,0.1842,...,27.28,136.50,1299.0,0.1396,0.5609,0.3965,0.18100,0.3792,0.10480,0.991388
2,849014,19.81,22.15,130.00,1260.0,0.09831,0.10270,0.14790,0.09498,0.1582,...,30.88,186.80,2398.0,0.1512,0.3150,0.5372,0.23880,0.2768,0.07615,0.993089
3,8511133,15.34,14.26,102.50,704.4,0.10730,0.21350,0.20770,0.09756,0.2521,...,19.08,125.10,980.9,0.1390,0.5954,0.6305,0.23930,0.4667,0.09946,0.959429
4,852763,14.58,21.53,97.41,644.8,0.10540,0.18680,0.14250,0.08783,0.2252,...,33.21,122.40,896.9,0.1525,0.6643,0.5539,0.27010,0.4264,0.12750,0.985463
5,854002,19.27,26.47,127.90,1162.0,0.09401,0.17190,0.16570,0.07593,0.1853,...,30.90,161.40,1813.0,0.1509,0.6590,0.6091,0.17850,0.3672,0.11230,0.993089
6,854253,16.74,21.59,110.10,869.5,0.09610,0.13360,0.13480,0.06018,0.1896,...,29.02,133.50,1229.0,0.1563,0.3835,0.5409,0.18130,0.4863,0.08633,0.993089
7,857373,13.64,16.34,87.21,571.8,0.07685,0.06059,0.01857,0.01723,0.1353,...,23.19,96.08,656.7,0.1089,0.1582,0.1050,0.08586,0.2346,0.08025,0.005192


#### 3. Update the output filter to keep only ID and prediction results
Let's change __output_filter__ to "$[0,-1]", indicating that when presenting the output, we only want to keep column 0 (the 'ID') and the last column (the inference result i.e. the probability of a given tumor to be malignant)

In [15]:
# start another transform job
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
    output_filter="$[0,-1]",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-01-22-08-47-979


.........................................[2024-06-01:22:15:33:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:15:33:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:15:33:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2024-06-01:22:15:33:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:15:33:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:22:15:33:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|

Now, let's inspect the output of the Batch Transform job in S3 again. It should show 2 columns: the ID and their corresponding probabilities of being malignant.

In [16]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1
0,844359,0.993089
1,84610002,0.991388
2,849014,0.993089
3,8511133,0.959429
4,852763,0.985463
5,854002,0.993089
6,854253,0.993089
7,857373,0.005192


create_model(role=role, image_uri=XGBOOST_IMAGE)In summary, we can use newly introduced 3 attributes - __input_filter__, __join_source__, __output_filter__ to 
1. Filter / select useful features from the input dataset. e.g. exclude ID columns.
2. Associate the prediction results with their corresponding input records.
3. Filter the original or joined results before saving to S3. e.g. keep ID and probability columns only.

## Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [17]:
sagemaker = boto3.client("sagemaker")

#model_name = "sagemaker-xgboost-2023-08-22-05-28-37-903"
model_name = job_name
print(model_name)


info = sagemaker.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

# Save our model to the Sagemaker Model Registry
create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2024-06-01-21-50-54
arn:aws:sagemaker:us-east-1:904981812149:model/xgb-2024-06-01-21-50-54


In [18]:
# Inspect Training Job Details
info

{'TrainingJobName': 'xgb-2024-06-01-21-50-54',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:904981812149:training-job/xgb-2024-06-01-21-50-54',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-21-50-54/xgb-2024-06-01-21-50-54/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'binary:logistic',
  'subsample': '0.8',
  'verbosity': '0'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:aucpr',
    'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\

## Create Endpoint Configuration

In [19]:
# Create Endpoint Configuration


# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'lab4-1-endpoint-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            
                            
instance_type = 'ml.m5.xlarge'

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")


Created EndpointConfig: arn:aws:sagemaker:us-east-1:904981812149:endpoint-config/lab4-1-endpoint-config2024-06-01-22-16-23


In [20]:
# Deploy our model to real-time endpoint

endpoint_name = 'lab4-1-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            


create_endpoint_response = sagemaker.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [21]:
# Wait for endpoint to spin up
import time 

sagemaker.describe_endpoint(EndpointName=endpoint_name)

while True:
    print("Getting Job Status")
    res = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    
    if state == "InService":
        print("Endpoint in Service")
        break
    elif state == "Creating":
        print("Endpoint still creating...")
        time.sleep(60)
    else:
        print("Endpoint Creation Error - Check Sagemaker Console")
        break

Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint in Service


In [22]:
# Invoke Endpoint

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
                            EndpointName=endpoint_name,
                            ContentType='text/csv',
                            Body=data_batch_noID.to_csv(header=None, index=False).strip('\n').split('\n')[0]
                            )
print(response['Body'].read().decode('utf-8'))

0.9930886626243591



In [23]:
# Examine Response Body

response

{'ResponseMetadata': {'RequestId': '32a93461-4a33-4724-9902-f5a7044d5cee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '32a93461-4a33-4724-9902-f5a7044d5cee',
   'x-amzn-invoked-production-variant': 'variant1',
   'date': 'Sat, 01 Jun 2024 22:20:25 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '19',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'variant1',
 'Body': <botocore.response.StreamingBody at 0x7fb3063c64d0>}

In [24]:
# Delete Endpoint

sagemaker.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '7e6bbe5d-f2a7-4d26-9ed5-773b4e156868',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7e6bbe5d-f2a7-4d26-9ed5-773b4e156868',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 01 Jun 2024 22:20:25 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

# Part 1: Set Up Model Group


First, import the required packages and set up the SageMaker Boto3 client.

In [25]:
import time 
import os
from sagemaker import get_execution_role, session
import boto3

#region = boto3.Session().region_name()
#role = get_execution_role()

sm_client = boto3.client('sagemaker', region_name= region)

Now create the Model Group.

In [26]:
import time

model_package_group_name = "xgboost-breast-cancer-detector-" + str(round(time.time()))
model_package_group_input_dict = {
    "ModelPackageGroupName" : model_package_group_name, 
    "ModelPackageGroupDescription" : "Group for versioned XGBoost models for breast cancer detection."
    }
create_model_package_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print("ModelPackageGroup Arn : {}".format(create_model_package_group_response['ModelPackageGroupArn']))

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:904981812149:model-package-group/xgboost-breast-cancer-detector-1717280426


Get the description of the model group

In [27]:
response = sm_client.describe_model_package_group(ModelPackageGroupName=model_package_group_name)
print(response)


{'ModelPackageGroupName': 'xgboost-breast-cancer-detector-1717280426', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:904981812149:model-package-group/xgboost-breast-cancer-detector-1717280426', 'ModelPackageGroupDescription': 'Group for versioned XGBoost models for breast cancer detection.', 'CreationTime': datetime.datetime(2024, 6, 1, 22, 20, 26, 51000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:904981812149:user-profile/d-bxoyca62lhab/shyam', 'UserProfileName': 'shyam', 'DomainId': 'd-bxoyca62lhab', 'IamIdentity': {'Arn': 'arn:aws:sts::904981812149:assumed-role/LabRole/SageMaker', 'PrincipalId': 'AROA5FNI6SO26R3SUIJA5:SageMaker'}}, 'ModelPackageGroupStatus': 'Completed', 'ResponseMetadata': {'RequestId': '5f5c7423-1894-4014-a10a-d57e2c3368ec', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5f5c7423-1894-4014-a10a-d57e2c3368ec', 'content-type': 'application/x-amz-json-1.1', 'content-length': '655', 'date': 'Sat, 01 Jun 2024 22:

# Part 2: Set Up Model Package

First, set up the parameter dictionary to pass to the create_model_package API operation.

In [28]:
# Specify the model source

# model_url is a ModelArtifacts or S3ModelArtifacts
model_url = "s3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-19-12-53/xgb-2024-06-01-19-12-53/output/model.tar.gz"
image_uri = '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1'


modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": image_uri,
	    "ModelDataUrl": model_url
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "XGBoost model trained to predict breast cancer (Benign and Malignant)",
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

create_model_package_input_dict

{'ModelPackageGroupName': 'xgboost-breast-cancer-detector-1717280426',
 'ModelPackageDescription': 'XGBoost model trained to predict breast cancer (Benign and Malignant)',
 'ModelApprovalStatus': 'PendingManualApproval',
 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
    'ModelDataUrl': 's3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-19-12-53/xgb-2024-06-01-19-12-53/output/model.tar.gz'}],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']}}

Then call the create_model_package API operation, passing in the parameter dictionary that just set up.

In [29]:
create_model_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_model_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-1:904981812149:model-package/xgboost-breast-cancer-detector-1717280426/1


Running describe model package using boto3

In [30]:
response = sm_client.describe_model_package(
    ModelPackageName = model_package_arn
    )
print(response)

{'ModelPackageGroupName': 'xgboost-breast-cancer-detector-1717280426', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:904981812149:model-package/xgboost-breast-cancer-detector-1717280426/1', 'ModelPackageDescription': 'XGBoost model trained to predict breast cancer (Benign and Malignant)', 'CreationTime': datetime.datetime(2024, 6, 1, 22, 20, 26, 589000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1', 'ImageDigest': 'sha256:cf81520a3b695293022793e292cf8bc3732b79231a6ebe1fb308086f6163a875', 'ModelDataUrl': 's3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-19-12-53/xgb-2024-06-01-19-12-53/output/model.tar.gz'}], 'SupportedContentTypes': ['text/csv'], 'SupportedResponseMIMETypes': ['text/csv']}, 'ModelPackageStatus': 'Completed', 'ModelPackageStatusDetails': {'ValidationStatuses': [], 'ImageScanStatuses': []}, 

# Part 3: Write the Model Card

Define an overview of model.

In [32]:

from sagemaker.model_card import (
    ModelCard,
    ModelOverview,
    ObjectiveFunction,
    Function,
    TrainingDetails,
    IntendedUses,
    BusinessDetails,
    EvaluationJob,
    AdditionalInformation,
    Metric,
    MetricGroup,
    ModelCardStatusEnum,
    ObjectiveFunctionEnum,
    FacetEnum,
    RiskRatingEnum,
    MetricTypeEnum,
    EvaluationMetricTypeEnum,
)

model_name = "xgb-2024-06-01-21-50-54" # this model is uploaded in the model registry

#model_name = "xgb-2024-06-01-19-12-53"  

model_overview = ModelOverview.from_model_name(
model_name=model_name,
sagemaker_session=sess,
model_description="Breast cancer Detection",
problem_type="Binary Classification",
algorithm_type="XGBoost", 
model_creator="Assignment4, USD.",
model_owner="AAI-540 Mlops, USD.",
)

print("Model id:", model_overview.model_id)
print("Model container image:", model_overview.inference_environment.container_image)
print("Model artifacts:", model_overview.model_artifact)

Model id: arn:aws:sagemaker:us-east-1:904981812149:model/xgb-2024-06-01-21-50-54
Model container image: ['683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1']
Model artifacts: ['s3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-21-50-54/xgb-2024-06-01-21-50-54/output/model.tar.gz']


### Define training details
To define your model's training details, we first define its objective function.

In [33]:
objective_function = ObjectiveFunction(
    function=Function(
        function=ObjectiveFunctionEnum.MINIMIZE,
        facet=FacetEnum.LOSS,
    ),
    notes="This is an objective-function",
)


Define training details using existing model overview, session, and objective function. Add any training observations here.

In [34]:
training_details = TrainingDetails.from_model_overview(
    model_overview=model_overview,
    sagemaker_session=sess,
    objective_function=objective_function,
    training_observations="Model-training-observations",
)

print(training_details.training_job_details.training_arn)
print(training_details.training_job_details.training_environment.container_image) 
print([{"name": i.name, "value": i.value} for i in training_details.training_job_details.training_metrics]) 

arn:aws:sagemaker:us-east-1:904981812149:training-job/xgb-2024-06-01-21-50-54
['683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1']
[{'name': 'validation:logloss', 'value': 0.13033999502658844}, {'name': 'train:logloss', 'value': 0.07124999910593033}]


Define evaluation details

In [35]:
# Define metric groups to describe metrics used  for a evaluation jobs
my_metric_group = MetricGroup(
name="binary classification metrics",
metric_data=[Metric(name="accuracy", type=MetricTypeEnum.NUMBER, value=0.5)]
)

In [36]:
# define evaluation details using evaluation metrics and datasets for each evaluation job
evaluation_details = [
    EvaluationJob(
        name="Example-evaluation-job",
        evaluation_observation="Evaluation-observations",
        datasets=['s3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/batch/batch_data_noID.csv'],
        metric_groups=[my_metric_group],
    )
]

### Define intended uses
Define the intended uses of the model, including the general purpose of the model and the use cases for which it was intended. 

In [37]:
intended_uses = IntendedUses(
    purpose_of_model="Designed to assist healthcare professionals in predicting breast cancer presence through clinical feature analysis.",
    intended_uses="Intended as a decision support tool for oncologists and radiologists to preliminarily assess breast cancer risk.",
    factors_affecting_model_efficiency="Affected by data quality, imaging variations, and demographics. Regular updates are recommended.",
    risk_rating=RiskRatingEnum.LOW,
    explanations_for_risk_rating="Rated low risk as it supplements, not replaces, medical judgment. Verification by clinical tests is advised.",
)

# Define additional information
additional_information = AdditionalInformation(
    ethical_considerations="Model usage must uphold patient confidentiality and adhere to healthcare regulations.",
    caveats_and_recommendations="Model is an aid not a substitute for professional diagnosis; continuous monitoring and validation are recommended.",
    custom_details={"Model Version": "1.0", "Data Source": "Clinical Patient Records"},
)


In [39]:
# define a model card
model_card_name = "Breast-Cancer-XGBoost-Card"
my_card = ModelCard(
    name=model_card_name,
    status=ModelCardStatusEnum.DRAFT,
    model_overview=model_overview,
    training_details=training_details,
    intended_uses=intended_uses,
    evaluation_details=evaluation_details,
    additional_information=additional_information,
    sagemaker_session=sess,
)
# Used definition to create a model card
my_card.create()

print(f"The model card named '{model_card_name}' has been successfully created")


INFO:sagemaker.model_card.model_card:Creating model card with name: Breast-Cancer-XGBoost-Card


The model card named 'Breast-Cancer-XGBoost-Card' has been successfully created


 Describe model card

In [40]:
response = sm_client.describe_model_card(
    ModelCardName= model_card_name,
)

print(response)

{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:904981812149:model-card/Breast-Cancer-XGBoost-Card', 'ModelCardName': 'Breast-Cancer-XGBoost-Card', 'ModelCardVersion': 1, 'Content': '{"model_overview": {"model_id": "arn:aws:sagemaker:us-east-1:904981812149:model/xgb-2024-06-01-21-50-54", "model_name": "xgb-2024-06-01-21-50-54", "model_description": "Breast cancer Detection", "problem_type": "Binary Classification", "algorithm_type": "XGBoost", "model_creator": "Assignment4, USD.", "model_owner": "AAI-540 Mlops, USD.", "model_artifact": ["s3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-21-50-54/xgb-2024-06-01-21-50-54/output/model.tar.gz"], "inference_environment": {"container_image": ["683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1"]}}, "intended_uses": {"purpose_of_model": "Designed to assist healthcare professionals in predicting breast cancer presence through clinical feature analysis.", "intended_uses":

###  Shutting down kernel for this notebook to release resources

In [41]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>